# MultiClass Classification in 10 Minutes with BERT-TensorFlow and SoftMax
- Based on Article  
  https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671

- Data Source:
  - Unzip files (only one time after downloading tar.gz file)  
  http://qwone.com/~jason/20Newsgroups/

  - Download Link:  
    http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz

In [40]:
from platform import python_version

print(python_version())

3.8.18


## Install Transformers Python Library to run it in CoLab

In [41]:
#!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval

## Mount Google Drive to Read Data & Model from Local Storage

In [42]:
device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA RTX A5000'

In [44]:
import gc
gc.collect()

8

In [45]:
dataset_name = 'S327Q02'
#sub_dataset = 'gender2'
train_data_file = '../dataset/extracted_files/'+dataset_name+'_train.csv'
output_model_name = '../models/G-SciEdBERT_model_'+dataset_name

In [46]:
train_df = pd.read_csv(train_data_file)
train_df.head()

,sentence,score
0,das bei ebbe weniger strom durch die turbinnen...,0.0
1,es könnten Überschwemmungen auftreten.,0.0
2,Die Turbinen sind sehr laut und liefern wenig ...,0.0
3,Man kann die Stromerzeugung bei einem Gezeiten...,0.0
4,Bei einem Gezeitenkraftwerk muss man sich auf ...,0.0


In [47]:
print('Unique comments in training: ', train_df.sentence.nunique() == train_df.shape[0])
print('Null values in training: ', train_df.isnull().values.any())
train_df = train_df.dropna()
print('Null values after drop in training: ', train_df.isnull().values.any())

Unique comments in training:  False
Null values in training:  False
Null values after drop in training:  False


In [48]:
train_df['label'] = pd.Categorical(train_df.score, ordered=False).codes
train_df['label'].unique()

array([0], dtype=int8)

In [49]:
mapLabels = pd.DataFrame(train_df.groupby(['score', 'label']).count())

#drop count column
mapLabels.drop(['sentence'], axis = 1, inplace = True)
label2Index = mapLabels.to_dict(orient='index')

print (f"label2Index :{label2Index}")
print (type(label2Index))
#print (f"index2Label :{index2Label}")

label2Index :{(0.0, 0): {}}
<class 'dict'>


In [50]:
index2label = {}

for key in label2Index:
  print (f"{key[1]} -> {key[0]}")
  index2label[key[1]] = key[0]

0 -> 0.0


In [51]:
label2Index = {v: k for k, v in index2label.items()}

print (f'label2Index: {label2Index}')
print (f'index2label: {index2label}')

label2Index: {0.0: 0}
index2label: {0: 0.0}


In [52]:
train_df.head()

,sentence,score,label
0,das bei ebbe weniger strom durch die turbinnen...,0.0,0
1,es könnten Überschwemmungen auftreten.,0.0,0
2,Die Turbinen sind sehr laut und liefern wenig ...,0.0,0
3,Man kann die Stromerzeugung bei einem Gezeiten...,0.0,0
4,Bei einem Gezeitenkraftwerk muss man sich auf ...,0.0,0


In [53]:
train_df.rename(columns = {'label' : 'LABEL_COLUMN', 'sentence' : 'DATA_COLUMN'}, inplace = True)

In [54]:
# Remoe Email address to avoid additional noise
train_df.DATA_COLUMN.replace(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', regex=True, inplace=True)

In [55]:
train_df = train_df[['LABEL_COLUMN','DATA_COLUMN']]

In [56]:
train_df.head()

,LABEL_COLUMN,DATA_COLUMN
0,0,das bei ebbe weniger strom durch die turbinnen...
1,0,es könnten Überschwemmungen auftreten.
2,0,Die Turbinen sind sehr laut und liefern wenig ...
3,0,Man kann die Stromerzeugung bei einem Gezeiten...
4,0,Bei einem Gezeitenkraftwerk muss man sich auf ...


In [57]:
train_df.count()

LABEL_COLUMN    686
DATA_COLUMN     686
dtype: int64

In [58]:
#splitSize = df.count() * .8
#splitSize

In [59]:
#people_copy = people.copy()
train = train_df.sample(frac=1, random_state=5)
#new_data = train.sample(frac=0.8, random_state=0)

#test = train_df.drop(train.index)

In [60]:
print (train.count())
unique_labels = np.unique(train["LABEL_COLUMN"].tolist())
label_counts = train["LABEL_COLUMN"].value_counts()
print(label_counts)
print(unique_labels)


LABEL_COLUMN    686
DATA_COLUMN     686
dtype: int64
LABEL_COLUMN
0    686
Name: count, dtype: int64
[0]


In [61]:
import pandas as pd
import numpy as np

# Function to calculate the Gini Coefficient
def gini_coefficient(array):
    """Calculate the Gini coefficient of a numpy array."""
    # All values are sorted and normalized (making the total equal to 1)
    array = array / array.sum()
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    return ((np.sum((2 * index - n - 1) * array)) / n)

# Calculate the Gini Coefficient for the label counts
gini = gini_coefficient(label_counts.values)
print(f"Gini Coefficient for the label distribution: {gini}")

Gini Coefficient for the label distribution: 0.0


In [62]:
validation_data_file = '../dataset/extracted_files/'+dataset_name+'_test.csv'
test_df = pd.read_csv(validation_data_file)
test_df.head()
print('Unique comments in testing: ', test_df.sentence.nunique() == test_df.shape[0])
print('Null values in testing: ', test_df.isnull().values.any())
test_df = test_df.dropna()
print('Null values after drop in testing: ', test_df.isnull().values.any())
test_df['score'] = pd.Categorical(test_df.score, ordered=True).codes
test_df['score'].unique()
test_df.rename(columns = {'score' : 'LABEL_COLUMN', 'sentence' : 'DATA_COLUMN'}, inplace = True)
test_df.DATA_COLUMN.replace(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', regex=True, inplace=True)
test_df = test_df[['LABEL_COLUMN','DATA_COLUMN']]
test = test_df.sample(frac=1, random_state=5)
print (test.count())
#unique_labels = np.unique(test_data["LABEL_COLUMN"].tolist())
#label_counts = test_data["LABEL_COLUMN"].value_counts()
#print(label_counts)
#print(unique_labels)

Unique comments in testing:  True
Null values in testing:  False
Null values after drop in testing:  False
LABEL_COLUMN    172
DATA_COLUMN     172
dtype: int64


In [63]:
uniqueLabels = train_df['LABEL_COLUMN'].unique()
print (f'Number of Labels: {len(uniqueLabels)},\nLabels:{uniqueLabels}')
sentences = list(train_df.DATA_COLUMN.values)

Number of Labels: 1,
Labels:[0]


## Load the Model
See Load and Save notebooks in this repository to understand how Transformers models cen be:
1. Downloaded
2. Stored Locally and
3. be used from Local Storage.

This should be interesting if you work in a cloud environment without Internet connection.

Here we tell the model that we whish to train on **20 label values** instead of the original 1 label (with 1 or 0 values) for which the original model was designed. This is why the test below tells us that we better should train this model. So, training it we will :-)

In [64]:
max_length = 512
model = TFBertForSequenceClassification.from_pretrained('bert-base-german-cased', num_labels=len(uniqueLabels))
#model = TFBertForSequenceClassification.from_pretrained('../models/G-SciEdBert', from_pt=True, num_labels=len(uniqueLabels))
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(sentences,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

loading configuration file https://huggingface.co/bert-base-german-cased/resolve/main/config.json from cache at C:\Users\el44163/.cache\huggingface\transformers\98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file https://huggingface.co/b

tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [65]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109081344 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 109,082,113
Trainable params: 109,082,113
Non-trainable params: 0
_________________________________________________________________


## Creating Input Sequences
We have two pandas Dataframe objects waiting for us to convert them into suitable objects for the BERT model. We will take advantage of the InputExample function that helps us to create sequences from our dataset. The InputExample function can be called as follows:

In [66]:
# transformers.InputExample
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

Now we will create two main functions:

1 — `convert_data_to_examples`: This will accept our train and test datasets and convert each row into an InputExample object.

2 — `convert_examples_to_tf_dataset`: This function will tokenize the InputExample objects, then create the required input format with the tokenized objects, finally, create an input dataset that we can feed to the model.

In [67]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  return train_InputExamples, validation_InputExamples

In [68]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train,
                                                                           test,
                                                                           'DATA_COLUMN',
                                                                           'LABEL_COLUMN')

In [69]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [70]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [71]:
print (str(type(DATA_COLUMN)) + ' ' + DATA_COLUMN)
print (str(type(LABEL_COLUMN)) + ' ' + LABEL_COLUMN)

<class 'str'> DATA_COLUMN
<class 'str'> LABEL_COLUMN


In [72]:
train.head(5)

,LABEL_COLUMN,DATA_COLUMN
293,0,"Wie viel Strom sie haben, hängt davon ab, ob E..."
683,0,Ein Kohlekraftwerk kann jeden Tag 24Stunden gl...
680,0,Das es eventuell Hochwasser geben könnte. Und ...
514,0,"Die Nachteile sind, dass das Wasser aus dem Fl..."
71,0,"Es wird nicht immer Strom geben, da Gezeiten a..."


In [73]:
%%time

train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


CPU times: total: 750 ms
Wall time: 756 ms


In [74]:
print (test.count())
unique_labels = np.unique(test["LABEL_COLUMN"].tolist())
label_counts = test["LABEL_COLUMN"].value_counts()
print(label_counts)
print(unique_labels)


LABEL_COLUMN    172
DATA_COLUMN     172
dtype: int64
LABEL_COLUMN
0    172
Name: count, dtype: int64
[0]


In [75]:
import pandas as pd
import numpy as np

# Function to calculate the Gini Coefficient
def gini_coefficient(array):
    """Calculate the Gini coefficient of a numpy array."""
    # All values are sorted and normalized (making the total equal to 1)
    array = array / array.sum()
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    return ((np.sum((2 * index - n - 1) * array)) / n)

# Calculate the Gini Coefficient for the label counts
gini = gini_coefficient(label_counts.values)
print(f"Gini Coefficient for the label distribution: {gini}")

Gini Coefficient for the label distribution: 0.0


## Configuring the BERT model and Fine-tuning
We will use Adam as our optimizer, CategoricalCrossentropy as our loss function, and SparseCategoricalAccuracy as our accuracy metric. Fine-tuning the model for 2 epochs will give us good accuracy, which is great.

In [76]:
%%time

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
44/44 [==============================] - 30s 328ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/2
44/44 [==============================] - 12s 274ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
CPU times: total: 37.4 s
Wall time: 42.5 s


In [37]:
#torch.save(model,output_model_name)
model.save_pretrained(output_model_name)

Configuration saved in ../models/G-SciEdBERT_model_combined\config.json
Model weights saved in ../models/G-SciEdBERT_model_combined\tf_model.h5


In [38]:
validation_data_file = '../dataset/extracted_files/'+dataset_name+'_test.csv'
test_df = pd.read_csv(validation_data_file)
test_df.head()
print('Unique comments in testing: ', test_df.sentence.nunique() == test_df.shape[0])
print('Null values in testing: ', test_df.isnull().values.any())
test_df = test_df.dropna()
print('Null values after drop in testing: ', test_df.isnull().values.any())
test_df['score'] = pd.Categorical(test_df.score, ordered=True).codes
test_df['score'].unique()
test_df.rename(columns = {'score' : 'LABEL_COLUMN', 'sentence' : 'DATA_COLUMN'}, inplace = True)
test_df.DATA_COLUMN.replace(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', regex=True, inplace=True)
test_df = test_df[['LABEL_COLUMN','DATA_COLUMN']]
test_data = test_df.sample(frac=1, random_state=5)
print (test_data.count())
unique_labels = np.unique(test_data["LABEL_COLUMN"].tolist())
label_counts = test_data["LABEL_COLUMN"].value_counts()
print(label_counts)
print(unique_labels)

Unique comments in testing:  False
Null values in testing:  False
Null values after drop in testing:  False
LABEL_COLUMN    9823
DATA_COLUMN     9823
dtype: int64
LABEL_COLUMN
0    4508
1    4281
2     607
5     256
6      90
7      43
3      21
4      17
Name: count, dtype: int64
[0 1 2 3 4 5 6 7]


In [39]:
pred_sentences= test_data["DATA_COLUMN"].tolist()
validation_labels = test_data["LABEL_COLUMN"].tolist()
tf_batch = tokenizer(pred_sentences, max_length=512, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

# Get index of predicted label for each sentence
predicted_labels = tf.argmax(tf_predictions, axis=1).numpy()

true_positives = 0

# output human readable label predictions
for i in range(len(pred_sentences)):
    predicted_label = predicted_labels[i]
    actual_label = validation_labels[i]
    if predicted_label == actual_label:
        true_positives+=1
accuracy = true_positives/len(pred_sentences)
print("Overall testing Accuracy:",accuracy )
        

    
#for i in range(len(pred_sentences)):
    #print(pred_sentences[i], ": \n", str(predicted_labels[i]) +" with score: "+ str(tf_predictions[i][predicted_labels[i]].numpy()))
    #print ("Actual Label:",str(validation_labels[i]) )

# Compute accuracy for each label
unique_labels = np.unique(validation_labels)
label_accuracies = {}

for label in unique_labels:
    correct_predictions = np.sum((predicted_labels == label) & (validation_labels == label))
    total_label_count = np.sum(validation_labels == label)
    
    accuracy = correct_predictions / total_label_count
    label_accuracies[label] = accuracy

print("Validation accuracy for each label:", label_accuracies)

ResourceExhaustedError: Exception encountered when calling layer "embeddings" "                 f"(type TFBertEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[9823,512,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResourceGather]

Call arguments received by layer "embeddings" "                 f"(type TFBertEmbeddings):
  • input_ids=tf.Tensor(shape=(9823, 512), dtype=int32)
  • position_ids=None
  • token_type_ids=tf.Tensor(shape=(9823, 512), dtype=int32)
  • inputs_embeds=None
  • past_key_values_length=0
  • training=False

In [32]:
#model = torch.load(output_model_name)
model_name = 'gelatin_gender2'
output_model_name = '../models/bert_model_'+model_name
#output_model_name = '../models/bert_model_ETS_CH_gelatin'
new_model = TFBertForSequenceClassification.from_pretrained(output_model_name)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer

loading configuration file ../models/bert_model_gelatin_gender2\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights fil

Training the model might take a while, so ensure you enabled the GPU acceleration from the Notebook Settings. After our training is completed, we can move onto making sentiment predictions.

## Making Predictions
I created a list of two reviews I created. The first one is a positive review, while the second one is clearly negative.

In [585]:
pred_sentences = ["The water is only stirring while the weight is falling. When the weight falls, the paddle will stop stirring."]

In [33]:
dataset_name = 'gelatin'
sub_dataset = 'gender1'
data_file = '../datasets/'+dataset_name+'/'+sub_dataset+'_test.csv'
df = pd.read_csv(data_file)
df.head()
print('Unique comments in training: ', df.sentence.nunique() == df.shape[0])
print('Null values in training: ', df.isnull().values.any())
df = df.dropna()
print('Null values after drop in training: ', df.isnull().values.any())
pred_sentences = list(df['sentence'])
actual_labels = list(df['score'])
print(len(pred_sentences))

Unique comments in training:  True
Null values in training:  False
Null values after drop in training:  False
86


We need to tokenize our reviews with our pre-trained BERT tokenizer. We will then feed these tokenized sequences to our model and run a final softmax layer to get the predictions. We can then use the argmax function to determine whether our sentiment prediction for the review is positive or negative. Finally, we will print out the results with a simple for loop. The following lines do all of these said operations:

In [34]:
import csv
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = new_model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

# Get index of predicted label for each sentence
pred_label = tf.argmax(tf_predictions, axis=1).numpy()
num_classes = tf_predictions.shape[1]

# output human readable label predictions
for i in range(len(pred_sentences)):
    print(pred_sentences[i], ": \n", str(pred_label[i]) +" with score: "+ str(tf_predictions[i][pred_label[i]].numpy()))
  #print(pred_sentences[i], ": \n", str(index2label[label[i]]) +" with score: "+ str(tf_predictions[i][label[i]].numpy()))
    print ()
with open('../outputfiles/'+model_name+'Model_'+sub_dataset+'_w_all_probs.csv', 'w',encoding="utf-8", newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Writing headers
    headers = ['Sentence', 'Actual Score', 'Predicted Score', 'Predicted Score Probability']
    headers += [f'Probability_Score_{i}' for i in range(num_classes)]
    csvwriter.writerow(headers)

    # Write data
    for i in range(len(pred_sentences)):
        sentence = pred_sentences[i]
        actual_score = actual_labels[i]  # or any other method to obtain the actual score
        bert_score = pred_label[i]
        probability = tf_predictions[i][pred_label[i]].numpy()
        probabilities = tf_predictions[i].numpy().tolist()

        # Write the row to the CSV file
        csvwriter.writerow([sentence, actual_score, bert_score, probability] + probabilities)


The initial temperatures and the masses of each should be measured To find the final temperature of the mixture, Qc+Qh = 0 Thus we can set Qc=-Qh, or mCdeltaTc= -mCdeltaTh We know the specific heat of water is a constant so it does not affect the equation deltaTc=Tfinal-Tinitial and deltaTh=Tfinal-Tinitial Since Tfinal will be the same for the final mixture, Tinitial must be known for each water Since mass is also a variable, it needs to be measured Thus the mass and the initial temperatures must be known : 
 4 with score: 0.5483377

We should ensure that when adding the two bowls of water together a temperature of atleast 40(C) is formed The temperature of two substances flows from warmer to colder, so as long as when the two fuse the temperature averages out at least at 40(C), there shouldn;t be an issue dissolving the Gelatin : 
 2 with score: 0.66361976

The original temperature of the gelatin and its amount and the amounts of hot water and cold water The amounts of hot and cold wa

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = new_model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
tf_predictions
tf.argmax(tf_predictions, axis=1).numpy()
index2label

## Debugging the Final Tensor Shape

In [ ]:
tf_predictions.shape

TensorShape([4, 20])

In [ ]:
for i in range(len(tf_predictions)):
  print (tf_predictions[i])

tf.Tensor(
[1.9084792e-04 3.4088054e-04 2.0756450e-04 8.6477579e-05 2.9989792e-04
 2.6212877e-04 1.9374983e-04 3.5850867e-04 1.5933276e-04 9.9522001e-01
 4.5725811e-04 8.2916165e-05 3.6863686e-04 1.2279976e-04 2.3117411e-04
 1.3213107e-04 3.1335116e-04 4.4700602e-04 2.9876176e-04 2.2643096e-04], shape=(20,), dtype=float32)
tf.Tensor(
[2.1327195e-04 4.0231278e-04 1.5158435e-04 1.2487071e-03 1.2888614e-03
 1.4331866e-03 4.6543666e-04 1.8467591e-04 4.1545741e-04 3.4950839e-04
 1.3036636e-03 9.8752570e-01 1.5377196e-03 3.4859296e-04 4.5611229e-04
 5.4127991e-04 2.9390829e-04 9.1435417e-04 2.4715456e-04 6.7858823e-04], shape=(20,), dtype=float32)
tf.Tensor(
[0.00853875 0.00138843 0.036679   0.01549945 0.00269004 0.00482676
 0.03897305 0.00894227 0.06801455 0.00392437 0.00970633 0.0075493
 0.00154941 0.44692346 0.00302002 0.18781705 0.02925803 0.00800841
 0.01692865 0.09976259], shape=(20,), dtype=float32)
tf.Tensor(
[0.00210652 0.32576597 0.08870737 0.05647071 0.01478916 0.450415
 0.0065489

In [ ]:
for i in range(len(tf_predictions)):
  print (str(tf_predictions[i][0]) + ' - ' + str(tf_predictions[i][1]))

tf.Tensor(0.00019084792, shape=(), dtype=float32) - tf.Tensor(0.00034088054, shape=(), dtype=float32)
tf.Tensor(0.00021327195, shape=(), dtype=float32) - tf.Tensor(0.00040231278, shape=(), dtype=float32)
tf.Tensor(0.008538753, shape=(), dtype=float32) - tf.Tensor(0.0013884273, shape=(), dtype=float32)
tf.Tensor(0.0021065164, shape=(), dtype=float32) - tf.Tensor(0.32576597, shape=(), dtype=float32)


In [59]:
for i in range(len(tf_predictions)):
  print(tf_predictions[i][label[i]].numpy())

0.9925568


Also, with the code above, you can predict as many reviews as possible.

# Congratulations

You have successfully built a transformers network with a pre-trained BERT model and achieved ~93% accuracy on the newsgroups classification analysis of the 20 Newsgroup reviews dataset! If you are curious about saving your model, I would like to direct you to the [Keras Documentation](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model). After all, to efficiently use an API, one must learn how to read and use the documentation.